In [23]:
# Developing a Convolutional Neural Network (CNN)
import numpy as np
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
import time

In [25]:
import pickle # Retrieving the variables that were saved in pickle earlier (from the processing part)

#  In order to read a particular variable 'X' & 'y' that was saved as pickle variable type load. X - features & y - labels 
pickle_in = open("X.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("y.pickle","rb")
y = pickle.load(pickle_in)

In [26]:
# Feature scaling, so that the model will be able to run in a faster way - normalising the images in terms of the features.
# X - features & Y - Labels
X=np.array(X/255.0)
Y=np.array(y)

In [27]:
LOG_DIR = f"{int(time.time())}"

In [28]:
# Using Keras tuner to perform hyperparameters optimization where Bayesian optimization is available
import keras_tuner
from keras_tuner.tuners import Hyperband, BayesianOptimization

In [29]:
def build_model(hp):  
    model = Sequential() # Developed the CNN layer by layer
    
    # Identifying the best no. of filters in the 1st convolution layer
    model.add(Conv2D(hp.Int("input units", min_value=32, max_value=128, step=16), (3 , 3), input_shape=(100, 100, 1)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    for i in range(hp.Int('n_layers', 1,4)): # Identifying the best no. of layers after the 1st convolution block
        model.add(Conv2D(hp.Int(f"conv_{i}_units", min_value=32, max_value=128, step=16), (3 , 3))) # Within these parameters
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        
    model.add(Flatten()) # Converts the 3D feature maps to 1D feature vectors
    model.add(Dense(hp.Choice('units',[16,32,48,64,80,96,112,128]))) # Finding out the best no. of neurons in this dense layer
    model.add(Dense(1)) # 1 output (COVID or NON-COVID)
    
    hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4]) # The best learning rate within this parameters
    opt = Adam(hp_learning_rate)
    model.add(Activation('sigmoid')) # Used for binary classification 0 & 1, more than 2 then use softmax
    model.compile(loss='binary_crossentropy',optimizer=opt, metrics=['accuracy'])

    return model   

In [31]:
# Max_trials implies the number of hyperparameter combinations that will be tested by the tuner
# Using validation accuaracy to determine the best model
tuner = BayesianOptimization(
        build_model,
        objective='val_accuracy',
        max_trials=6)

In [32]:
# The types of hyperparameters to be tuned and the boundaries 
tuner.search_space_summary()

Search space summary
Default search space size: 5
input units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 128, 'step': 16, 'sampling': None}
n_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 4, 'step': 1, 'sampling': None}
conv_0_units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 128, 'step': 16, 'sampling': None}
units (Choice)
{'default': 16, 'conditions': [], 'values': [16, 32, 48, 64, 80, 96, 112, 128], 'ordered': True}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [33]:
# Train on 10720 and 1340 images
from sklearn.model_selection import train_test_split
trainX,testX,trainY,testY = train_test_split(X,Y,test_size=0.1111)

In [34]:
# Reflects the no. of labels in the validation set
len(testY)

1340

In [35]:
# Reflects the no. of images in the training set
len(trainX)

10720

In [36]:
# Searching for the best parameters under build model, for only 3 epochs
tuner.search(trainX,trainY,
             epochs=3,
             validation_data=(testX,testY))

Trial 6 Complete [00h 03m 35s]
val_accuracy: 0.9171642065048218

Best val_accuracy So Far: 0.9611940383911133
Total elapsed time: 00h 24m 32s
INFO:tensorflow:Oracle triggered exit


In [37]:
best_hyperparameters = tuner.get_best_hyperparameters(1)[0]
print(best_hyperparameters)

In [38]:
# The best hyperparameters for the model
tuner.get_best_hyperparameters()[0].values

{'input units': 32,
 'n_layers': 1,
 'conv_0_units': 128,
 'units': 16,
 'learning_rate': 0.001,
 'conv_1_units': 80}

In [39]:
# Showing further details of the best model
n_best_models = tuner.get_best_models(num_models=2)
print(n_best_models[0].summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 98, 98, 32)        320       
_________________________________________________________________
activation (Activation)      (None, 98, 98, 32)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 49, 49, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 47, 47, 128)       36992     
_________________________________________________________________
activation_1 (Activation)    (None, 47, 47, 128)       0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 23, 23, 128)       0         
_________________________________________________________________
flatten (Flatten)            (None, 67712)             0

In [40]:
model = tuner.hypermodel.build(best_hyperparameters)

In [41]:
# Re-train the best model, which was identified earlier
history = model.fit(trainX,trainY,
             epochs=15,
             validation_data=(testX,testY))

Epoch 1/15
335/335 [==============================] - 74s 220ms/step - loss: 0.3192 - accuracy: 0.8789 - val_loss: 0.2072 - val_accuracy: 0.9239
Epoch 2/15
335/335 [==============================] - 73s 217ms/step - loss: 0.1822 - accuracy: 0.9324 - val_loss: 0.2084 - val_accuracy: 0.9216
Epoch 3/15
335/335 [==============================] - 73s 219ms/step - loss: 0.1169 - accuracy: 0.9552 - val_loss: 0.0944 - val_accuracy: 0.9672
Epoch 4/15
335/335 [==============================] - 75s 223ms/step - loss: 0.0695 - accuracy: 0.9760 - val_loss: 0.0885 - val_accuracy: 0.9649
Epoch 5/15
335/335 [==============================] - 73s 218ms/step - loss: 0.0483 - accuracy: 0.9827 - val_loss: 0.0686 - val_accuracy: 0.9716
Epoch 6/15
335/335 [==============================] - 74s 222ms/step - loss: 0.0280 - accuracy: 0.9904 - val_loss: 0.0841 - val_accuracy: 0.9649
Epoch 7/15
335/335 [==============================] - 73s 218ms/step - loss: 0.0225 - accuracy: 0.9915 - val_loss: 0.0569 - val_ac

In [42]:
# Determining the accuracy
_,accuracy = model.evaluate(testX,testY,verbose=0)
print('Accuracy: %.3f' % (accuracy*100))

Accuracy: 98.507


In [43]:
# Saving the model
model.save("my_model")

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: my_model\assets
